In [1]:
import pickle
import pathlib

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score

In [2]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'
print(DATA_DIR)

c:\Users\luis\Documents\6\CSRoundPrediction\data


In [3]:
clean_data_path = DATA_DIR / 'processed' / 'csgo_clean.pkl'

In [4]:
with open(clean_data_path, 'rb') as file:
    data = pickle.load(file)

Checking if the data was properly read.

In [5]:
model_data = data.copy()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122410 entries, 0 to 122409
Data columns (total 76 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   time_left                     122410 non-null  float64
 1   ct_score                      122410 non-null  float64
 2   t_score                       122410 non-null  float64
 3   bomb_planted                  122410 non-null  bool   
 4   ct_health                     122410 non-null  float64
 5   t_health                      122410 non-null  float64
 6   ct_armor                      122410 non-null  float64
 7   t_armor                       122410 non-null  float64
 8   ct_money                      122410 non-null  float64
 9   t_money                       122410 non-null  float64
 10  ct_helmets                    122410 non-null  float64
 11  t_helmets                     122410 non-null  float64
 12  ct_defuse_kits                122410 non-nul

-------------------------------

# Splitting the data

The data will be split into be split into 2 sets: training and testing. The training set will be used to train the model and the testing set will be used to evaluate the model.

In [10]:
X = model_data.drop(columns=['round_winner']).copy()
y = model_data['round_winner'].copy()

Creating the test set with 20% of the data.

In [11]:
x_train_val, x_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Creating the train and validation sets with 80% of the data.

In [12]:
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.2, random_state=42)

In [13]:
x_train.shape, x_val.shape, x_test.shape

((78342, 75), (19586, 75), (24482, 75))

--------------------------------

# Choosing the model

In [16]:
def calculate_model_quality(model,x_val, y_val):
    y_predicted = model.predict(x_val)
    accuracy = accuracy_score(y_val, y_predicted)
    f1 = f1_score(y_val, y_predicted)
    
    print(f'Accuracy: {accuracy:.2f}')
    print(f'F1: {f1:.2f}')
    

### LogisticRegression

In [33]:
# 0.75 But it doesn't converge
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1750
                           )
model.fit(x_train, y_train)
calculate_model_quality(model, x_val, y_val)

Accuracy: 0.75
F1: 0.75


/home/louis/Documents/6/ml/CSRoundPrediction/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### RandomForestClassifier

In [34]:
# 0.87 
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=100,
    max_depth=30,
    random_state=42,
    min_samples_split=2,
)

model.fit(x_train, y_train)
calculate_model_quality(model, x_val, y_val)

Accuracy: 0.87
F1: 0.87


### DecisionTreeClassifier

In [38]:
# 0.81
from sklearn import tree

model = tree.DecisionTreeClassifier(
    random_state=42,
    max_depth=60,
)

model.fit(x_train, y_train)
calculate_model_quality(model, x_val, y_val)

Accuracy: 0.81
F1: 0.81


### GradientBoostingClassifier

In [36]:
# 0.85
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(
    n_estimators=75, 
    learning_rate=0.8, 
    max_depth=30,
    random_state=42
)
model.fit(x_train, y_train)
calculate_model_quality(model, x_val, y_val)

Accuracy: 0.85
F1: 0.85


### KNeighborsClassifier

In [37]:
# 0.75
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=5)
model.fit(x_train, y_train)
calculate_model_quality(model, x_val, y_val)

Accuracy: 0.75
F1: 0.75


### Mini Models!

This model was sugested by the professor, and it's idea is to split the data into several parts. Then, going through chunks of three parts, three models are trained and the majority vote is taken as the final prediction.

The cell bellow is a showcase of the model. Using just 1 split, the data will not be split into chunks, and the result should be the same as the 
[RandomForestClassifier](#randomforestclassifier)

In [13]:
import MiniModels
from sklearn.ensemble import RandomForestClassifier

Splitmodel = MiniModels.MiniModels()
model_used = RandomForestClassifier(
    n_estimators=100,
    max_depth=30,
    random_state=42,
    min_samples_split=2,
)

Splitmodel.fit(x_train, y_train, model_used, n_splits=1)
calculate_model_quality(Splitmodel, x_val, y_val)

Self.parts:
Part 0 size: (78342, 76), time_left: 175.0 - 0.03
Accuracy: 0.87
F1: 0.87


In [25]:
import MiniModels
from sklearn.ensemble import RandomForestClassifier

Splitmodel = MiniModels.MiniModels()
# model_used = RandomForestClassifier(random_state=42, max_depth=25, n_estimators=100)
model_used = RandomForestClassifier(
    n_estimators=100,
    max_depth=30,
    random_state=42,
)

Splitmodel.fit(x_train, y_train, model_used, 5)
calculate_model_quality(Splitmodel, x_val, y_val)

Self.parts:
Part 0 size: (15668, 76), time_left: 175.0 - 174.91
Part 1 size: (15668, 76), time_left: 174.91 - 107.19
Part 2 size: (15668, 76), time_left: 107.18 - 74.95
Part 3 size: (15668, 76), time_left: 74.95 - 39.45
Part 4 size: (15670, 76), time_left: 39.45 - 0.03
Models trained
Accuracy: 0.82
F1: 0.82


In [29]:
import MiniModels
from sklearn import tree

Splitmodel = MiniModels.MiniModels()
model_used = tree.DecisionTreeClassifier(random_state=42, max_depth=20)

Splitmodel.fit(x_train, y_train, model_used, 1)
calculate_model_quality(Splitmodel, x_val, y_val)

Self.parts:
Part 0 size: (15668, 76), time_left: 175.0 - 174.91
Part 1 size: (15668, 76), time_left: 174.91 - 107.19
Part 2 size: (15668, 76), time_left: 107.18 - 74.95
Part 3 size: (15668, 76), time_left: 74.95 - 39.45
Part 4 size: (15670, 76), time_left: 39.45 - 0.03
Models trained
Accuracy: 0.77
F1: 0.77


In [20]:
import MiniModels
from sklearn.ensemble import GradientBoostingClassifier

Splitmodel = MiniModels.MiniModels()
model_used = GradientBoostingClassifier(
    n_estimators=75, 
    learning_rate=0.8, 
    max_depth=30,
    random_state=42
)

Splitmodel.fit(x_train, y_train, model_used, 5)
calculate_model_quality(Splitmodel, x_val, y_val)

Self.parts:
Part 0 size: (15668, 76), time_left: 175.0 - 174.91
Part 1 size: (15668, 76), time_left: 174.91 - 107.19
Part 2 size: (15668, 76), time_left: 107.18 - 74.95
Part 3 size: (15668, 76), time_left: 74.95 - 39.45
Part 4 size: (15670, 76), time_left: 39.45 - 0.03
Models trained
Accuracy: 0.80
F1: 0.80


--------------------------------

# Certification

Training and evaluating the best model with the test set.

In [26]:
# 0.87 
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=100,
    max_depth=30,
    random_state=42,
    min_samples_split=2,
)

model.fit(x_train_val, y_train_val)
calculate_model_quality(model, x_test, y_test)

Accuracy: 0.88
F1: 0.88


Evualating model quality using a DummyClassifier.

In [37]:
# Most frequent class
most_frequent_class = y_train_val.value_counts()
print(most_frequent_class) # Print the Distribution of the classes

# Get the most frequent class
most_frequent_class = most_frequent_class.idxmax()

# Create an array with the same shape as y_test and fill it with the most frequent class
y_predicted = np.full_like(y_test, fill_value=most_frequent_class)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_predicted)
print()
print(f'Accuracy: {accuracy:.2f}')

round_winner
0    49928
1    48000
Name: count, dtype: int64

Accuracy: 0.51


Model was certified with 0.88 accuracy and beat the DummyClassifier.

```markdown
👍
```

----

# Preparing the Deployment

Using the elected model, the data will be trained again, but now with the full dataset.

In [44]:
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=30,
    random_state=42,
    min_samples_split=2,
)

model.fit(X, y) # Train the model with the full dataset

RandomForestClassifier(max_depth=30, random_state=42)

In [45]:
# Save the model
model_path = DATA_DIR / 'models' / 'csgo_model.pkl'

with open(model_path, 'wb') as file:
    pickle.dump(model, file)